In [ ]:
#matplotlib notebook
import numpy as np
import math
import random
import pyJHTDB
import h5py
import time as tt
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import filters
import skimage.morphology as morphology
from skimage.morphology import square
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from matplotlib import cm
import scipy.io as sio
import time
import matplotlib.animation as animation
# load shared library
lTDB = pyJHTDB.libJHTDB()
#initialize webservices
lTDB.initialize()

#Add token
auth_token  = "edu.jhu.pha.turbulence.testing-201311" #use your personal authorisation token for full-access to JHTDB
lTDB.add_token(auth_token)

dataset = 'transition_bl'

spatialInterp  = 0  # no spatial interpolation
temporalInterp = 0  # no time interpolation
FD4NoInt       = 40 # 4th-order FD, no spatial interpolation

In [ ]:
# Database domain size and number of grid points
x_min =   30.2185
x_max = 1000.0650
y_min =    0.0036
y_max =   26.4880
z_min =    0.0000
z_max =  240.0000
d99i  =    0.9648
d99f  =   15.0433

nx = 3320
ny =  224
nz = 2048

# Database time duration
Ti = 0
Tf = Ti + 1175
dt = 0.25

# Create surface
nix = round(nx / 8)
niz = round(nz / 8)
niy = round(ny / 50)
x = np.linspace(x_min, x_max, nix)
z = np.linspace(z_min, z_max, niz)
y = np.linspace(y_min, y_max, niy); #d99i

[X,Z] = np.meshgrid(x,z)
print(niz,nix,niy,nix*niy*niz)

In [ ]:
# Get the velocity at each point
time = Ti #Time of the data
ui=np.zeros((nix,niz,niy,3))
duidxj=np.zeros((nix,niz,niy,9))

for j in range(niy):
    print('Read y:',y[j])
    start = tt.time()
    points  = np.zeros((nix,niz,3))
    points[:,:,0] = X.transpose()
    points[:,:,1] = y[j]
    points[:,:,2] = Z.transpose()

    # 3D array with single precision values
    points = np.array(points,dtype='float32')

    print('Requesting velocity at {0} points...'.format(nix*niz*1))
    result = lTDB.getData(time, points, data_set = 'transition_bl',
            sinterp = spatialInterp, tinterp = temporalInterp,
            getFunction = 'getVelocity')
    end = tt.time()
    print('   '+str(end - start)+' seconds')

    start = tt.time()
    print('Requesting velocity gradients at {0} points...'.format(nix*niz*1))
    result_grad = lTDB.getData(time, points, data_set = 'transition_bl',
            sinterp = FD4NoInt, tinterp = temporalInterp,
            getFunction = 'getVelocityGradient')
    end = tt.time()
    print('   '+str(end - start)+' seconds')
    ui[0:nix,0:niz,j,0:3]=result[0:nix,0:niz,0:3]
    duidxj[0:nix,0:niz,j,0:9]=result_grad[0:nix,0:niz,0:9]

In [ ]:
#Save in matlab format   
sio.savemat('data_t0.mat',{'x':x,'y':y,'z':z,'ui':ui,'duidxj':duidxj})